##  Digital Watermaking by Dugad algorithm

Задача: поставить подпись на изображение и иметь возможность ее обнаружить. Для этого используем wavelet-преобразование.
Для начала сделаем необходимые включения:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pywt._multidim import dwt2,idwt2
from PIL import Image
import numpy as np

Далее зафиксируем значение параметра альфа (коэффициент в преобразовании). Ключом будет являться переменная w,
которая в нашем случае будет просто генерироваться по заданному seed. Она будет создана сразу после получения размеров изображения.

In [10]:
alpha = 0.3
np.random.seed(645)

Возьмем изображение '2.bmp' (вообще наш алгоритм позволяет работать и с JPEG). Сделаем  wavelet-преобразование

In [12]:
a = Image.open('2.bmp')
a = a.convert('YCbCr')
pix = a.load() # Доступ к пикселям
    
size_x  = a.size[0]
size_y = a.size[1]
data = np.zeros((size_x,size_y)) # Массив с пикселями

w = np.random.randn(size_x,size_y) # Для генерации подписи

for i in range(size_x):
    for j in range(size_y):
        data[i,j] = pix[i,j][0] # Работаем с яркостью

cA, (cH, cV, cD) = dwt2(data,'db4') # Непосредственно преобразование

## Encryption

Далее прибавим к коэффициентам, которые больше T1, некоторое значение. Параметр alpha был подобран экспериментально. Преобразование выполняется согласно алгоритму.

In [14]:
T1 = 40
for i in range(cV.shape[0]):
    for j in range(cV.shape[1]):
        if cV[i,j] > T1:
            cV[i,j] += alpha*w[i,j]*abs(cV[i,j])
for i in range(cH.shape[0]):
    for j in range(cH.shape[1]):
        if cH[i,j] > T1:
            cH[i,j] += alpha*w[i,j]*abs(cH[i,j])
for i in range(cD.shape[0]):
    for j in range(cD.shape[1]):
        if cD[i,j] > T1:
            cD[i,j] += alpha*w[i,j]*abs(cD[i,j])

Выполним обратное преобразование с измененнными матрицами коэффициентов.

In [16]:
ret = idwt2((cA,(cH,cV,cD)),'db4')
for i in range(size_x):
    for j in range(size_y):
        pix[i,j] = (int(ret[i,j]),pix[i,j][1],pix[i,j][2])
a.save('well.jpg')

## Validation

Для валидации существующей подписи откроем сохраненное изображение с подписью

In [17]:
test = Image.open('well.jpg')
test = test.convert('YCbCr')
pixels = test.load()
    
size_x  = test.size[0]
size_y = test.size[1]
data = np.zeros((size_x,size_y))

for i in range(size_x):
    for j in range(size_y):
        data[i,j] = pixels[i,j][0]
cA, (cH, cV, cD) = dwt2(data,'db6')

Теперь установим порог T2=50 и будем вычислять корреляцию

In [18]:
T2 = 50
z = 0
m = 0
for i in range(cV.shape[0]):
    for j in range(cV.shape[1]):
        if cV[i,j] > T2:
            m += 1
            z += cV[i,j]*w[i,j]
for i in range(cH.shape[0]):
    for j in range(cH.shape[1]):
        if cH[i,j] > T2:
            m += 1
            z += cH[i,j]*w[i,j]
for i in range(cD.shape[0]):
    for j in range(cD.shape[1]):
        if cD[i,j] > T2:
            m += 1
            z += cD[i,j]*w[i,j]
z = z / m
s = 0
m = 0
for i in range(cV.shape[0]):
    for j in range(cV.shape[1]):
        if cV[i,j] > T2:
            m += 1
            s += abs(cV[i,j])
for i in range(cH.shape[0]):
    for j in range(cH.shape[1]):
        if cH[i,j] > T2:
            m += 1
            s += abs(cH[i,j])
for i in range(cD.shape[0]):
    for j in range(cD.shape[1]):
        if cD[i,j] > T2:
            m += 1
            s += abs(cD[i,j])
s = alpha * s/ (2 * m) 

Итак, у нас есть два значения: z и s. Как утверждает алгоритм, если z>s, то заданная подпись есть на изображении

In [19]:
z>s

True

Ради интереса изменим подпись и выполним валидацию еще раз. Дабы не использовать клетки снизу вверх, просто воспользуемся методом Copy-Paste

In [21]:
np.random.seed(13)
w = np.random.randn(size_x,size_y)

In [22]:
test = Image.open('well.jpg')
test = test.convert('YCbCr')
pixels = test.load()
    
size_x  = test.size[0]
size_y = test.size[1]
data = np.zeros((size_x,size_y))

for i in range(size_x):
    for j in range(size_y):
        data[i,j] = pixels[i,j][0]
cA, (cH, cV, cD) = dwt2(data,'db6')
T2 = 50
z = 0
m = 0
for i in range(cV.shape[0]):
    for j in range(cV.shape[1]):
        if cV[i,j] > T2:
            m += 1
            z += cV[i,j]*w[i,j]
for i in range(cH.shape[0]):
    for j in range(cH.shape[1]):
        if cH[i,j] > T2:
            m += 1
            z += cH[i,j]*w[i,j]
for i in range(cD.shape[0]):
    for j in range(cD.shape[1]):
        if cD[i,j] > T2:
            m += 1
            z += cD[i,j]*w[i,j]
z = z / m
s = 0
m = 0
for i in range(cV.shape[0]):
    for j in range(cV.shape[1]):
        if cV[i,j] > T2:
            m += 1
            s += abs(cV[i,j])
for i in range(cH.shape[0]):
    for j in range(cH.shape[1]):
        if cH[i,j] > T2:
            m += 1
            s += abs(cH[i,j])
for i in range(cD.shape[0]):
    for j in range(cD.shape[1]):
        if cD[i,j] > T2:
            m += 1
            s += abs(cD[i,j])
s = alpha * s/ (2 * m) 

Мы изменили подпись, но не меняли изображение. Ожидаемый результат: z < s

In [23]:
z < s

True

Таким образом, то чего нет - необнаружено. А то что есть - найдено. 
В ходе тестов установлено, что алгоритм устойчив к сжатию и изменению яркости изображения. Однако не усточив при обрезании изоображения.